# Initial Plan for U.S. Patent Phrase to Phrase Matching

## Objective
Predict semantic similarity score between anchor and target phrases, given context (CPC code). Metric: Pearson correlation coefficient. Target: Gold medal >=0.87.

## High-Level Strategy
1. **Environment Setup:** Verify GPU availability and install necessary packages (e.g., PyTorch with CUDA, transformers).
2. **Data Loading & EDA:** Load train/test CSV, explore distributions of scores, phrase lengths, unique contexts, sample examples.
3. **Preprocessing:** Handle text cleaning, possibly tokenize with BERT tokenizer. Use context to disambiguate.
4. **Modeling:** Start with a simple baseline using TF-IDF + Ridge regression. Then move to transformer-based models like Sentence-BERT or DeBERTa for embeddings, followed by a regressor. Use cross-validation (e.g., 5-fold) to estimate performance.
5. **Feature Engineering:** Extract features like phrase overlap, lengths, context-specific embeddings.
6. **Training & Validation:** Train with early stopping, monitor CV score. Aim for CV >0.86 to target medal.
7. **Inference & Submission:** Generate predictions on test, save to submission.csv.

## Milestones for Expert Review
- After EDA: Understand data quirks.
- After baseline model: Check if simple approach gets >0.8 CV.
- After transformer baseline: Target >0.85 CV.
- After FE and tuning: Push to medal territory.

## Risks & Mitigations
- Data leakage: Ensure CV splits don't mix contexts inappropriately.
- Overfitting: Use OOF predictions for analysis.
- Compute limits: Use GPU, subsample for quick iterations.

Next: Insert code cell to check GPU and load data.

In [1]:
import os
import sys
import subprocess
import shutil
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Check GPU availability
print('Checking GPU...')
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout if result.returncode == 0 else 'GPU not available')

# Install PyTorch with CUDA 12.1 (following best practices)
def pip_install(*args):
    print('Installing:', *args)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Uninstall existing torch if any
subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'torch', 'torchvision', 'torchaudio'], check=False)

# Clean stray directories
stray_dirs = [
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch'
]
for d in stray_dirs:
    if os.path.exists(d):
        print(f'Removing {d}')
        shutil.rmtree(d, ignore_errors=True)

# Install PyTorch cu121
pip_install('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# Create constraints
Path('constraints.txt').write_text(
    'torch==2.4.1\\n'
    'torchvision==0.19.1\\n'
    'torchaudio==2.4.1\\n')

# Install other packages
pip_install('install', '-c', 'constraints.txt',
    'transformers==4.44.2', 'accelerate==0.34.2',
    'datasets==2.21.0', 'evaluate==0.4.2',
    'sentencepiece', 'scikit-learn',
    '--upgrade-strategy', 'only-if-needed')

# Verify installation
import torch
print(f'torch: {torch.__version__} built CUDA: {getattr(torch.version, "cuda", None)}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
else:
    print('ERROR: GPU not available - exiting competition')
    # Note: In real scenario, we'd use exit tool, but for now, assert
    assert torch.cuda.is_available(), 'GPU required'

# Load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(f'Train shape: {train.shape}')
print(f'Test shape: {test.shape}')

# Basic EDA
print('\nTrain columns:', train.columns.tolist())
print('\nScore distribution:')
print(train['score'].value_counts().sort_index())
print(f'\nScore stats: mean={train["score"].mean():.4f}, std={train["score"].std():.4f}')

print('\nSample data:')
print(train.head())

print('\nUnique contexts:')
print(train['context'].nunique(), 'unique contexts')
print(train['context'].value_counts().head())

print('\nPhrase lengths:')
train['anchor_len'] = train['anchor'].str.len()
train['target_len'] = train['target'].str.len()
print(f'Anchor avg len: {train["anchor_len"].mean():.1f}, Target avg len: {train["target_len"].mean():.1f}')

# Plot score distribution
plt.figure(figsize=(8,4))
sns.histplot(train['score'], bins=20, kde=True)
plt.title('Score Distribution')
plt.show()

# Sample examples by score
for score in sorted(train['score'].unique()):
    sample = train[train['score'] == score].sample(1)
    print(f'\nScore {score}: anchor="{sample["anchor"].iloc[0]}", target="{sample["target"].iloc[0]}", context={sample["context"].iloc[0]}')

Checking GPU...
Fri Sep 26 15:31:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     128MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------------------

Installing: install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 324.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 159.4 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 341.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 389.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 KB 493.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 KB 526.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 429.0 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 KB 541.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of <Python from Requires-Python> to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.

The conflict is caused by:
    accelerate 0.34.2 depends on torch>=1.10.0
    The user requested (constraint) torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict



ERROR: Cannot install accelerate==0.34.2 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


CalledProcessError: Command '['/usr/bin/python3.11', '-m', 'pip', 'install', '-c', 'constraints.txt', 'transformers==4.44.2', 'accelerate==0.34.2', 'datasets==2.21.0', 'evaluate==0.4.2', 'sentencepiece', 'scikit-learn', '--upgrade-strategy', 'only-if-needed']' returned non-zero exit status 1.

In [2]:
# Fix installation and continue EDA
import subprocess
import sys
from pathlib import Path

# Verify torch installation
import torch
print(f'torch: {torch.__version__} built CUDA: {getattr(torch.version, "cuda", None)}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
else:
    raise AssertionError('GPU required')

# Install remaining packages without strict constraints to avoid conflict
def pip_install(*args):
    print('Installing:', *args)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

pip_install('install',
    'transformers', 'accelerate',
    'datasets==2.21.0', 'evaluate==0.4.2',
    'sentencepiece', 'scikit-learn')

# Now load data and EDA (pandas, numpy, etc. are pre-installed)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(f'Train shape: {train.shape}')
print(f'Test shape: {test.shape}')

# Basic EDA
print('\nTrain columns:', train.columns.tolist())
print('\nScore distribution:')
print(train['score'].value_counts().sort_index())
print(f'\nScore stats: mean={train["score"].mean():.4f}, std={train["score"].std():.4f}')

print('\nSample data:')
print(train.head())

print('\nUnique contexts:')
print(train['context'].nunique(), 'unique contexts')
print(train['context'].value_counts().head())

print('\nPhrase lengths:')
train['anchor_len'] = train['anchor'].str.len()
train['target_len'] = train['target'].str.len()
print(f'Anchor avg len: {train["anchor_len"].mean():.1f}, Target avg len: {train["target_len"].mean():.1f}')

# Plot score distribution
plt.figure(figsize=(8,4))
sns.histplot(train['score'], bins=20, kde=True)
plt.title('Score Distribution')
plt.show()

# Sample examples by score
for score in sorted(train['score'].unique()):
    sample = train[train['score'] == score].sample(1)
    print(f'\nScore {score}: anchor="{sample["anchor"].iloc[0]}", target="{sample["target"].iloc[0]}", context={sample["context"].iloc[0]}')

torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q
Installing: install transformers accelerate datasets==2.21.0 evaluate==0.4.2 sentencepiece scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 196.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 KB 257.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 KB 496.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 KB 415.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 481.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 199.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 417.6 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 KB 467.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 509.9 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.19.1+cu121 requires torch==2.4.1, but you have torch 2.8.0 which is incompatible.
torchaudio 2.4.1+cu121 requires torch==2.4.1, but you have torch 2.8.0 which is incompatible.


Train shape: (32825, 5)
Test shape: (3648, 4)

Train columns: ['id', 'anchor', 'target', 'context', 'score']

Score distribution:
score
0.00     6774
0.25    10306
0.50    11068
0.75     3634
1.00     1043
Name: count, dtype: int64

Score stats: mean=0.3619, std=0.2589

Sample data:
                 id           anchor                        target context  \
0  378b8322a01b88db  obstacle course  obstacle position trajectory     B60   
1  f43c824b3d939294  hardware blocks                       housing     G06   
2  c4ef1b4f79424269         collator           collation apparatus     H04   
3  f8702c95bfa192fe     engage clamp              disconnect clamp     H01   
4  5c24dd3361ba190d   oven batteries                     batteries     C10   

   score  
0   0.50  
1   0.25  
2   0.75  
3   0.25  
4   0.50  

Unique contexts:
106 unique contexts
context
H04    1962
H01    1956
G01    1633
A61    1312
F16     970
Name: count, dtype: int64

Phrase lengths:
Anchor avg len: 16.0, Target avg


Score 0.0: anchor="drive balls", target="tennis ball", context=F16

Score 0.25: anchor="finger actuator", target="pivot pin", context=B23

Score 0.5: anchor="donor plasmid", target="vector", context=A61

Score 0.75: anchor="end pins", target="support pins", context=H01

Score 1.0: anchor="output center", target="center output", context=B62
